In [ ]:
import ccsfg
import FactorGraphGeneration as FG
import ccsinnercode as ccsic

# Set up Graph

In [ ]:
# Graph Parameters
L = 16                  # Number of sections/sub-blocks
vl = 16                 # Length of each coded sub-block
ml = 2**vl              # Length of each section of m

# Instantiate CCS Graph
class CCSGraph(ccsfg.SystematicEncoding):
    def __init__(self, seclength=ml):
        self.__Check2VarEdges = []
        self.__Check2VarEdges.append([1, 2, 3])
        self.__Check2VarEdges.append([4, 5, 6])
        self.__Check2VarEdges.append([7, 8, 9])
        self.__Check2VarEdges.append([10, 11, 12])
        self.__Check2VarEdges.append([1, 7, 13])
        self.__Check2VarEdges.append([2, 10, 14])
        self.__Check2VarEdges.append([4, 8, 15])
        self.__Check2VarEdges.append([5, 11, 16])
        super().__init__(self.__Check2VarEdges, [1, 2, 4, 5, 7, 8, 10, 11], seclength)
        self.__DepthFromRoot = 32

    def getmaxdepth(self):
        return self.__DepthFromRoot

# Simulation Parameters

In [ ]:
# Set up multiple access scenario
# Notation has been chosen to match https://arxiv.org/pdf/2010.04364.pdf
Ka = 25                 # Number of active users
w = 128                 # Payload size of each active user (per user message length)
N = 38400               # Total number of channel uses (real d.o.f)

# Simulation Parameters
numAmpIter = 6          # Number of AMP iterations
listSize = 4 * Ka       # List size retained for each section after AMP converges
BPonOuterGraph = 1      # Indicates whether to perform BP on outer code.  If 0, AMP uses Giuseppe's uninformative prior
EbNodB = 2.4            # Energy per bit in decibels
maxSims = 10            # Number of Simulations to Run

# Prepare Simulation
EbNo = 10**(EbNodB/10)  # Eb/No in linear scale
P = 2*w*EbNo/N          # transmit power
std = 1                 # Noise standard deviation
Phat = N*P/L            # Power estimate
errorRate = 0.0         # track error rate across simulations

# Run Simulation

In [ ]:
# Initialize CCS-AMP Graph
Graph = CCSGraph()


# Run CCS-AMP maxSims times
for sims in range(maxSims):
    
    # Reset the graph
    Graph.reset()
    
    # Set up Inner Encoder/Decoder
    InnerCode = ccsic.DenseInnerCode(L, ml, N, P, std, Ka)
    
    # Generate random messages for Ka active users
    txBits = np.random.randint(low=2, size=(Ka, w))
    
    # Outer LDPC Encoder
    x = Graph.encodesignal(txBits)
    
    # Inner CS Encoder
    x = InnerCode.Encode(Phat, x)
    
    # Transmit x over channel
    y = (x + (np.random.randn(N, 1) * σ_n)).reshape(-1, 1)

    # Inner CS Decoder
    xHt, _ = InnerCode.Decode(y, numAmpIter, BPonOuterGraph, graph)
    
    # Outer LDPC Decoder (Message Disambiguation)
    txBitsHt = FG.decoder(Graph, xHt, listSize)
    
    # Calculate PUPE
    errorRate += (Ka - FG.numbermatches(txBits, txBitsHt)) / (Ka * maxSims)

# Display Simulation Results
print("Per user probability of error = %3.6f" % errorRate)